In [1]:
import pandas as pd

In [2]:
movies = pd.read_csv('http://bit.ly/imdbratings')
movies.head()

,star_rating,title,content_rating,genre,duration,actors_list
0,9.3,The Shawshank Redemption,R,Crime,142,"[u'Tim Robbins', u'Morgan Freeman', u'Bob Gunt..."
1,9.2,The Godfather,R,Crime,175,"[u'Marlon Brando', u'Al Pacino', u'James Caan']"
2,9.1,The Godfather: Part II,R,Crime,200,"[u'Al Pacino', u'Robert De Niro', u'Robert Duv..."
3,9.0,The Dark Knight,PG-13,Action,152,"[u'Christian Bale', u'Heath Ledger', u'Aaron E..."
4,8.9,Pulp Fiction,R,Crime,154,"[u'John Travolta', u'Uma Thurman', u'Samuel L...."


ตอนนี้เรามาโฟกัสที่คอลัมน์ content_rating เพื่อหา missing value

In [3]:
movies.content_rating.isnull().sum()

3

In [4]:
movies[movies.content_rating.isnull()]

,star_rating,title,content_rating,genre,duration,actors_list
187,8.2,Butch Cassidy and the Sundance Kid,NaN,Biography,110,"[u'Paul Newman', u'Robert Redford', u'Katharin..."
649,7.7,Where Eagles Dare,NaN,Action,158,"[u'Richard Burton', u'Clint Eastwood', u'Mary ..."
936,7.4,True Grit,NaN,Adventure,128,"[u'John Wayne', u'Kim Darby', u'Glen Campbell']"


ต่อมาลองนับจำนวนความถี่ของข้อมูลที่มีความแตกต่างกันในคอลัมน์นี้ดู

In [5]:
movies.content_rating.value_counts()

R            460
PG-13        189
PG           123
NOT RATED     65
APPROVED      47
UNRATED       38
G             32
NC-17          7
PASSED         7
X              4
GP             3
TV-MA          1
Name: content_rating, dtype: int64

ต่อมาเราจะนำค่า NaN ไปกำหนดแทนที่ข้อมูลที่มีค่า NOT RATED ใน DataFrame ที่ทำการ slice ตรงๆดังนี้

In [6]:
import numpy as np

In [7]:
movies[movies.content_rating == 'NOT RATED'].content_rating = np.nan

/home/sutthisak/anaconda3/envs/learning-python/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


จะเห็นว่ามี SettingWithCopyWarning เกิดขึ้นจากการกำหนดค่าให้ series ข้างต้น และข้อมูลไม่ได้ถูกแทนที่จริงๆ

In [8]:
movies.content_rating.isnull().sum()

3

นั่นคือถ้าเราจะทำการแทนที่ข้อมูลใน DataFrame ที่มีการ slice ออกมานั้นเราต้อง slice ข้อมูลด้วย loc method ตามที่ warning แสดงออกมาข้างต้น

In [9]:
movies.loc[movies.content_rating == 'NOT RATED', 'content_rating'] = np.nan

In [10]:
movies.content_rating.isnull().sum()

68

มาลองสร้าง DataFrame ชุดใหม่จากข้อมูลเดิมกัน

In [11]:
top_movies = movies.loc[movies.star_rating >= 9, :]
top_movies

,star_rating,title,content_rating,genre,duration,actors_list
0,9.3,The Shawshank Redemption,R,Crime,142,"[u'Tim Robbins', u'Morgan Freeman', u'Bob Gunt..."
1,9.2,The Godfather,R,Crime,175,"[u'Marlon Brando', u'Al Pacino', u'James Caan']"
2,9.1,The Godfather: Part II,R,Crime,200,"[u'Al Pacino', u'Robert De Niro', u'Robert Duv..."
3,9.0,The Dark Knight,PG-13,Action,152,"[u'Christian Bale', u'Heath Ledger', u'Aaron E..."


ทดลองเปลี่ยนเวลาของแถวที่ 0 ดังนี้

In [12]:
top_movies.loc[0, 'duration'] = 150

/home/sutthisak/anaconda3/envs/learning-python/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


อ้าว!!!! ใช้ loc method แล้วทำไมยังเจอ SettingWithCopyWarning อยู่ล่ะ

In [13]:
top_movies

,star_rating,title,content_rating,genre,duration,actors_list
0,9.3,The Shawshank Redemption,R,Crime,150,"[u'Tim Robbins', u'Morgan Freeman', u'Bob Gunt..."
1,9.2,The Godfather,R,Crime,175,"[u'Marlon Brando', u'Al Pacino', u'James Caan']"
2,9.1,The Godfather: Part II,R,Crime,200,"[u'Al Pacino', u'Robert De Niro', u'Robert Duv..."
3,9.0,The Dark Knight,PG-13,Action,152,"[u'Christian Bale', u'Heath Ledger', u'Aaron E..."


แต่มันดันกำหนดค่าลงไปได้ ช่างชวนทำให้สับสนงงงวยจริงๆ

ปัญหาจริงๆคือตัว pandas ไม่รู้ว่าตัว top_movies นั้นเป็น วิวที่อ้างอิง movies หรือ สำเนาของ movies ดังนั้นถ้าเราจะทำสำเนาข้อมูลจาก DataFrame ออกมา เราต้องประกาศให้ชัดๆลงไปเลยว่า เราจะทำการสำเนาน๊ะ เราไม่ได้ทำการสร้างวิว ได้ดังนี้

In [14]:
top_movies = movies.loc[movies.star_rating >= 9, :].copy()
top_movies

,star_rating,title,content_rating,genre,duration,actors_list
0,9.3,The Shawshank Redemption,R,Crime,142,"[u'Tim Robbins', u'Morgan Freeman', u'Bob Gunt..."
1,9.2,The Godfather,R,Crime,175,"[u'Marlon Brando', u'Al Pacino', u'James Caan']"
2,9.1,The Godfather: Part II,R,Crime,200,"[u'Al Pacino', u'Robert De Niro', u'Robert Duv..."
3,9.0,The Dark Knight,PG-13,Action,152,"[u'Christian Bale', u'Heath Ledger', u'Aaron E..."


In [15]:
top_movies.loc[0, 'duration'] = 150
top_movies

,star_rating,title,content_rating,genre,duration,actors_list
0,9.3,The Shawshank Redemption,R,Crime,150,"[u'Tim Robbins', u'Morgan Freeman', u'Bob Gunt..."
1,9.2,The Godfather,R,Crime,175,"[u'Marlon Brando', u'Al Pacino', u'James Caan']"
2,9.1,The Godfather: Part II,R,Crime,200,"[u'Al Pacino', u'Robert De Niro', u'Robert Duv..."
3,9.0,The Dark Knight,PG-13,Action,152,"[u'Christian Bale', u'Heath Ledger', u'Aaron E..."


แล้วเราก็ไม่เจอ SettingWithCopyWarning อีกต่อไป ;P